In [ ]:
import nbformat
from nbconvert import MarkdownExporter, PDFExporter
from nbconvert.preprocessors import ExecutePreprocessor
from pyprojroot import here

In [ ]:
import yaml
from pyprojroot import here


def read_mkdocs():
    with open(here() / "mkdocs.yml", "r+") as f:
        f = "".join(l for l in f.readlines())
        mkdocs_config = yaml.safe_load(f)
    return mkdocs_config


mkdocs_config = read_mkdocs()

In [ ]:
nav = mkdocs_config["nav"]
docroot = here() / "docs"


In [ ]:
# The goal here is to flatten the tree structure into a list of 2-tuples,
# where the title is the first element and the filename is the second element.
accumulator = []

def parse_navigation(nav, accumulator):
    """
    Collect all .ipynb files to be parsed
    into a list of 2-tuples.
    """
    for item in nav:
        if isinstance(item, dict):
            for k, v in item.items():
                if isinstance(v, list):
                    parse_navigation(v, accumulator)
                if isinstance(v, str):
                    accumulator.append((k, v))
            
    return accumulator
    
accumulated = parse_navigation(nav, accumulator)
accumulated = accumulated[2:]
accumulated

In [ ]:
def read_markdown(fpath):
    with open(fpath, "r+") as f:
        md = f.read()
    return md

In [ ]:
read_markdown(here() / "docs/index.md")

In [ ]:
def read_notebook(fpath):
    with open(fpath, "r+") as f:
        nb = nbformat.reads(f.read(), as_version=4)
    return nb

In [ ]:
nb = read_notebook(here() / "docs/practical/io.ipynb")
# nb["cells"]

In [ ]:
nb.keys()

In [ ]:
nb.metadata

This is how we are going to approach the problem. We are going to create ONE GIANT NOTEBOOK
and use the PDFExporter to do exporting.

God Bless Me as I attempt this...

In [ ]:
def md2nbcell(md):
    """Convert markdown to Jupyter notebook cell."""
    data = {
        "cell_type": "markdown",
        "metadata": {},
        "source": md
    }
    cell = nbformat.NotebookNode(**data)
    return cell

In [ ]:
def compile_code_cells(accumulated):
    cells = []
    for title, file in accumulated:
        fpath = here() / "docs" / file
        titlecell = md2nbcell(f"# {title}")
        cells.append(titlecell)

        if file.endswith(".md"):
            md = read_markdown(fpath)
            cell = md2nbcell(md)
            cells.append(cell)
        elif file.endswith(".ipynb"):
            nb = read_notebook(fpath)
            cells.extend(nb.cells)
    return cells

In [ ]:
cells = compile_code_cells(accumulated)

In [ ]:
def make_compiled_notebook(cells):
    metadata = {'kernelspec': {'display_name': 'nams', 'language': 'python', 'name': 'nams'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.7.7'}}
    
    compiled_nb = nbformat.v4.new_notebook()
    compiled_nb.metadata = metadata
    compiled_nb.cells = cells
    return compiled_nb

In [ ]:
compiled_nb = make_compiled_notebook(cells)

In [ ]:
def to_pdf(nb):
    ep = ExecutePreprocessor(timeout=600, kernel_name='nams')
    ep.preprocess(nb)
    pdf_exporter = PDFExporter()
    body, resources = pdf_exporter.from_notebook_node(nb)
    return body, resources

In [ ]:
body, resources = to_pdf(compiled_nb)

In [ ]:
def write_pdf(body, fpath):
    with open(fpath, "wb") as f:
        f.write(body)

In [ ]:
write_pdf(body, "output.pdf")